# Task1_ANN

Index_X = FSR_for_force, FSR_for_coord

Index_y = force, x_coord, y_coord

Data = Splited by Time

## Run result

https://wandb.ai/seokjin/FSR-prediction/groups/FSR_Trainable_2023-08-17_11-30-03/workspace?workspace=user-seokjin

## Experiment id

FSR_Trainable_2023-08-17_11-30-03

## Best metric (RMSE)



0.6783

## Fixed Hyperparameter

model: ANN / hidden size: 8 / num layer: 1

optimizer: NAdam

In [1]:
import ray.tune
import ray.air
import ray.air.integrations.wandb
import ray.tune.schedulers
from fsr_trainable import FSR_Trainable
import ray.tune.search
import ray.tune.search.optuna

tuner = ray.tune.Tuner(
    trainable=ray.tune.with_resources(
        FSR_Trainable, {'cpu':2},
    ),
    tune_config=ray.tune.TuneConfig(
        num_samples=4,
        reuse_actors=True,
        scheduler=ray.tune.schedulers.PopulationBasedTraining(
            time_attr='time_total_s',
            perturbation_interval=5,
            metric='metric',
            mode='min',
            hyperparam_mutations={
                'model':['fsr_model.ANN'],
                'model_args':{
                    'hidden_size':[8],
                    'num_layer':[1],
                },
                'criterion':['torch.nn.MSELoss'],
                'optimizer':[
                    'torch.optim.NAdam',
                ],
                'optimizer_args':{
                    'lr':ray.tune.loguniform(1e-5, 1e-1),
                },
                'imputer':['sklearn.impute.SimpleImputer'],
                'imputer_args':{
                    'strategy':['mean', 'median'],
                },
                'scaler':[ 
                    'sklearn.preprocessing.StandardScaler',
                    'sklearn.preprocessing.MinMaxScaler',
                    'sklearn.preprocessing.RobustScaler',
                ],
                'index_X': [['FSR_for_force', 'FSR_for_coord']],
                'index_y': [['force', 'x_coord', 'y_coord']],
                'data_loader': ['fsr_data.get_index_splited_by_time']
            },
        ),
    ), 
    run_config=ray.air.RunConfig(
        callbacks=[
            ray.air.integrations.wandb.WandbLoggerCallback(project='FSR-prediction'),
        ],
        checkpoint_config=ray.air.CheckpointConfig(
            num_to_keep=3,
            checkpoint_score_attribute='metric',
            checkpoint_score_order='min',
            checkpoint_frequency=5,
            checkpoint_at_end=True,
        ),
    ), 
)

In [2]:
results = tuner.fit()

2023-08-17 11:30:06,353	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2023-08-17 11:30:08,846	INFO tune.py:226 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Tuner(...)`.


2023-08-17 11:30:08,923	INFO wandb.py:320 -- Already logged into W&B.


Trial name,date,done,hostname,iterations_since_restore,mae_coord,mae_force,mape_coord,mape_force,metric,node_ip,pid,rmse_coord,rmse_force,time_since_restore,time_this_iter_s,time_total_s,timestamp,tmae_coord,tmae_force,tmape_coord,tmape_force,training_iteration,trial_id,trmse_coord,trmse_force
FSR_Trainable_fbe7d_00000,2023-08-17_11-31-20,False,DESKTOP-0P789CI,1,0.903458,232.833,0.281962,1.26185,7.22889,172.26.215.93,245855,1.98782,440.042,0.709959,0.709959,21.781,1692239480,1.57443,1.30331,1.40656e+15,1.40945e+15,31,fbe7d_00000,3.73139,3.4975
FSR_Trainable_fbe7d_00001,2023-08-17_11-31-21,False,DESKTOP-0P789CI,7,0.063806,32.2362,0.00947381,2.95631e+15,0.417216,172.26.215.93,245852,0.10621,82.4744,5.15632,0.776073,26.0608,1692239481,0.149442,0.114768,1.32163e+13,2.03217e+14,35,fbe7d_00001,0.212779,0.204437
FSR_Trainable_fbe7d_00002,2023-08-17_11-30-45,False,DESKTOP-0P789CI,1,0.402373,111.741,0.0983136,1.07774e+17,0.198186,172.26.215.93,245852,0.691733,211.73,0.903552,0.903552,16.9101,1692239445,0.0798743,0.0424389,1.32982e+13,4.54902e+13,23,fbe7d_00002,0.126992,0.0711936
FSR_Trainable_fbe7d_00003,2023-08-17_11-31-07,False,DESKTOP-0P789CI,7,0.201835,82.7553,0.0305562,3.73973e+15,0.652004,172.26.215.93,245852,0.268964,200.505,5.40261,1.11391,21.4582,1692239467,0.352534,0.193137,6.19984e+12,3.97045e+14,29,fbe7d_00003,0.419101,0.232903


(_WandbLoggingActor pid=246064) wandb: Currently logged in as: seokjin. Use `wandb login --relogin` to force relogin
(_WandbLoggingActor pid=246064) wandb: wandb version 0.15.8 is available!  To upgrade, please run:
(_WandbLoggingActor pid=246064) wandb:  $ pip install wandb --upgrade
(_WandbLoggingActor pid=246064) wandb: Tracking run with wandb version 0.15.4
(_WandbLoggingActor pid=246064) wandb: Run data is saved locally in /home/seokj/ray_results/FSR_Trainable_2023-08-17_11-30-03/FSR_Trainable_fbe7d_00003_3_2023-08-17_11-30-09/wandb/run-20230817_113027-fbe7d_00003
(_WandbLoggingActor pid=246064) wandb: Run `wandb offline` to turn off syncing.
(_WandbLoggingActor pid=246064) wandb: Syncing run FSR_Trainable_fbe7d_00003
(_WandbLoggingActor pid=246064) wandb: ⭐️ View project at https://wandb.ai/seokjin/FSR-prediction
(_WandbLoggingActor pid=246064) wandb: 🚀 View run at https://wandb.ai/seokjin/FSR-prediction/runs/fbe7d_00003
2023-08-17 11:30:42,900	INFO pbt.py:808 -- 

[PopulationBas

TuneError: The Ray Tune run failed. Please inspect the previous error messages for a cause. After fixing the issue, you can restart the run from scratch or continue this run. To continue this run, you can use `tuner = Tuner.restore("/home/seokj/ray_results/FSR_Trainable_2023-08-17_11-30-03", trainable=...)`.